In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('lyrics.csv')

In [3]:
df.tail(10)

,Unnamed: 0,Title,Lyrics,English Translations
4220,4220,O Bedardeya,"['Lyrics', 'Pyar jhootha tha jataya hi kyun', ...","['Translation', 'Why did you express your love..."
4221,4221,Tere Vaaste,"['Lyrics', 'Tere vaaste falak se main chaand l...","['Translation', ""I'll bring you the moon from ..."
4222,4222,Phir Aur Kya Chahiye,"['Lyrics', 'Badle tere maahi', 'Laake joh koyi...","['Translation', 'In exchange for you my belove..."
4223,4223,Tum Kya Mile,"['Lyrics', 'Berange the din, berangi shaamein'...","['Translation', 'My days and evenings were wit..."
4224,4224,What Jumka,"['Lyrics', 'Pretty lady, pretty lady, pretty l...","['Translation', 'Pretty lady, pretty lady, pre..."
4225,4225,Chal Tere Ishq Mein,"['Lyrics', 'Joh bhi faisla hai tera woh mujhko...","['Translation', 'Tell me your decision whateve..."
4226,4226,Khairiyat,"['Lyrics', 'Aye do jahaan ke malik sun le meri...","['Translation', 'O Lord of both the worlds, li..."
4227,4227,Dil Jhoom,"['Lyrics', 'Yeh shaukiye shararat yeh nafasat ...","['Translation', 'This style, mischief, exquisi..."
4228,4228,Kudmayi,"['Lyrics', 'Salma sitaron wali shagna di shubh...","['Translation', 'O beloved, the starry auspici..."
4229,4229,Ve Kamleya,"['Lyrics', 'Ve kamleya, ve kamleya', 'Ve kamle...","['Translation', 'O crazy one, O crazy one', 'O..."


In [4]:

import nltk
nltk.download('punkt')
import nltk
nltk.download('vader_lexicon')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chand\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Chand\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [5]:
import pandas as pd
import re

# Read the CSV file into a DataFrame
df = pd.read_csv('lyrics.csv')

# Define the combined tokenization and stemming function
def hinglish_tokenization_and_stemming(text):
    tokens = re.findall(r'\b\w+\b', text)
    stemmed_tokens = []
    for token in tokens:
        if token.endswith(('ing', 'ed', 'yon', 'ta', 'te', 'ti', 'le', 'li', 'lo', 'ya', 'ye', 'yi', 'yo', 'ra', 're', 'ri', 'ro', 'na', 'ne', 'ni', 'no')):
            token = token[:-2]
        elif token.endswith(('ta', 'te', 'ti', 'ya', 'ye', 'yi', 'yo', 'na', 'ne', 'ni', 'no')):
            token = token[:-2]
        elif token.endswith(('a', 'e', 'i', 'o', 'u')):
            token = token[:-1]
        stemmed_tokens.append(token)
    return ' '.join(stemmed_tokens)

# Apply the function to the Lyrics column
df['Lyrics'] = df['Lyrics'].apply(hinglish_tokenization_and_stemming)



In [6]:
df['Lyrics'] = df['Lyrics'].apply(hinglish_tokenization_and_stemming)

In [7]:
df['Title'] = df['Title'].fillna('Unknown').astype(str)

In [8]:
df['Song_Type'] = df['Title'].apply(lambda title: 'sad' if 'sad' in title.lower() else 'party' if 'party' in title.lower() else 'normal')

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk import download

# Download the VADER lexicon if not already downloaded
download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Chand\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [10]:
sia = SentimentIntensityAnalyzer()

In [11]:
tfidvector = TfidfVectorizer(analyzer='word', stop_words='english')
matrix = tfidvector.fit_transform(df['Lyrics'])
similarity = cosine_similarity(matrix)


In [12]:
matrix.shape

(4230, 15897)

In [13]:
def recommendation(song_title, song_type=None):
    # Get the index of the song
    idx = df[df['Title'] == song_title].index[0]
    
    # Compute distances
    distances = sorted(list(enumerate(similarity[idx])), reverse=True, key=lambda x: x[1])
    
    # Filter songs by song type if provided
    songs = []
    for m_id in distances[1:21]:
        song_row = df.iloc[m_id[0]]
        if song_type is None or song_row['Song_Type'] == song_type:  # Filter by song type
            songs.append(song_row['Title'])
    
    return songs

In [14]:
recommendation('O Bedardeya')

['Bairi Piya',
 'Piya Piya O Piya',
 'Agar Tum Saath Ho',
 'Chandni Raat Hai',
 'Sona Kitna Sona Hai',
 'Yaar Piya',
 'Hum To Deewane Hue Yaar',
 'Aaya Aaya',
 'Mohabbat Ki Nahin Jaati',
 'Mere Yaar Ki Shaadi Hai',
 'Tu Rootha Dil Toota',
 'Yeh Mera Dil',
 'Mujhe Neend Na Aaye',
 'Dil Yeh Bekarar Kyun Hai',
 'Baatein Hawa Hai',
 'Yari Hai Imaan Mera',
 'Kyon Na Hum Milke Pyar',
 'Achha Sila Diya Tune Mere Pyar Ka',
 'Latoo',
 'Ishq Kamaal']

In [15]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))